In [ ]:
#AIDI 1002: Final Project
#Group Members: Gokul Shrivastav(200579267) & Omoririe Awolala()
#Paper Link: https://arxiv.org/abs/1901.11196
#GitHub:https://github.com/jasonwei20/eda_nlp

In [5]:
#Cloning the repository
!git clone https://github.com/jasonwei20/eda_nlp.git
%cd eda_nlp/code


C:\Users\Gokul Shrivastav\eda_nlp\code\eda_nlp\code


fatal: destination path 'eda_nlp' already exists and is not an empty directory.


In [13]:
!pip install nltk
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to C:\Users\Gokul
[nltk_data]     Shrivastav\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to C:\Users\Gokul
[nltk_data]     Shrivastav\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Gokul
[nltk_data]     Shrivastav\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [9]:
#Running the model
from eda import eda

# Giving an example sentence
sentence = "I want this Semester to end "

# Generate the augmented sentences
augmented_sentences = eda(sentence, num_aug=4)#setting how aggresive the augmentations are
augmented_sentences

C:\Users\Gokul Shrivastav\eda_nlp\code\eda_nlp\code\eda.py:177: SyntaxWarning: "is not" with 'str' literal. Did you mean "!="?
  words = [word for word in words if word is not '']


['semester want this i to end',
 'i want this oddment semester to end',
 'i this semester to end',
 'i privation this semester to end',
 'i want this semester to end ']

In [11]:
#Giving multiple example sentences and also changing the augmentation attributes
sentences = [
    "Canada is a Great country",
    "The food is amazing.",
    "Highly recommend to everyone.",
]

aug_data = []
for s in sentences:
    aug_data.extend(eda(s, num_aug=3))

for s in aug_data:
    print(s)

canada is a great country
canada personify is a great country
canada is a great land
canada is a great country
the food for thought is amazing
the food personify is amazing
the food is amazing
the food is amazing 
highly extremely recommend to everyone
highly urge to everyone
highly recommend everyone to
highly recommend to everyone 


In [ ]:
#We can see above that formation of sentences is better with changed attributes
#For making a contribution we will be eveluate the model on a pubicily available dataset on IMDB movie reviews

In [17]:
#Loading the new dataset and printing the first 10 values

from datasets import load_dataset
import pandas as pd

dataset = load_dataset("imdb")
train_df = pd.DataFrame(dataset['train'])

#Reducing dataset size for quicker experimentation
train_df = train_df.sample(n=1000, random_state=42).reset_index(drop=True)

train_df[['text', 'label']].head(10)


,text,label
0,"Dumb is as dumb does, in this thoroughly unint...",0
1,I dug out from my garage some old musicals and...,1
2,After watching this movie I was honestly disap...,0
3,This movie was nominated for best picture but ...,1
4,Just like Al Gore shook us up with his painful...,1
5,"Unfortunately, this movie is so bad. The origi...",0
6,Altered Species starts one Friday night in Los...,0
7,'The Luzhin Defence' is a movie worthy of anyo...,1
8,This film's trailer interested me enough to wa...,0
9,The only reason I watched this film was becaus...,0


In [19]:
#Applying EDA to the training data

augmented_texts = []
augmented_labels = []

for index, row in train_df.iterrows():
    original_sentence = row['text']
    label = row['label']
    
    #Applying EDA: generate 3 augmented versions per sentence
    augmented_sentences = eda(original_sentence, num_aug=3)
    
    augmented_texts.extend(augmented_sentences)
    augmented_labels.extend([label]*len(augmented_sentences))

#Combining original and augmented data
augmented_df = pd.DataFrame({
    'text': train_df['text'].tolist() + augmented_texts,
    'label': train_df['label'].tolist() + augmented_labels
})

augmented_df.sample(5)


,text,label
4563,this title seems more like a filming exercise ...,0
755,Steven Seagal....how could you be a part of su...,0
2402,this is another the curse of east asian cinema...,0
863,I thought it was an extremely clever film. I w...,1
4429,if you like star wars trek cum see where they ...,1


In [21]:
#Training a classifier on the original and Augmented data

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

#Performing TF-IDF vectorization
vectorizer = TfidfVectorizer(stop_words='english', max_features=3000)

#Splitting original data
X_train, X_test, y_train, y_test = train_test_split(
    train_df['text'], train_df['label'], test_size=0.3, random_state=42)

X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

#Training on original data
clf_orig = LogisticRegression(max_iter=200)
clf_orig.fit(X_train_vec, y_train)

y_pred_orig = clf_orig.predict(X_test_vec)
print("Original Data Performance:\n", classification_report(y_test, y_pred_orig))


Original Data Performance:
               precision    recall  f1-score   support

           0       0.76      0.78      0.77       145
           1       0.79      0.77      0.78       155

    accuracy                           0.77       300
   macro avg       0.77      0.77      0.77       300
weighted avg       0.77      0.77      0.77       300



In [23]:

#Splitting the augmented data
X_aug_train, _, y_aug_train, _ = train_test_split(
    augmented_df['text'], augmented_df['label'], test_size=0.2, random_state=42)

X_aug_train_vec = vectorizer.fit_transform(X_aug_train)
X_test_vec = vectorizer.transform(X_test)  

#Training on augmented data
clf_aug = LogisticRegression(max_iter=200)
clf_aug.fit(X_aug_train_vec, y_aug_train)

y_pred_aug = clf_aug.predict(X_test_vec)
print("Augmented Data Performance:\n", classification_report(y_test, y_pred_aug))


Augmented Data Performance:
               precision    recall  f1-score   support

           0       1.00      0.99      0.99       145
           1       0.99      1.00      0.99       155

    accuracy                           0.99       300
   macro avg       0.99      0.99      0.99       300
weighted avg       0.99      0.99      0.99       300



In [25]:
train_df.to_csv("original_data.csv", index=False)
augmented_df.to_csv("augmented_data.csv", index=False)


In [ ]:
#Contribution Summary
#In this project, we investigated the performance of Jason Wei and Kai Zou's Easy Data Augmentation (EDA) technique on a new dataset,the IMDB movie reviews dataset, for binary sentiment classification.  The original EDA work did not use this dataset in its trials, therefore this is an extension of their methods to a different domain.
#We replicated the original EDA augmentation workflow and used it on a subset of the IMDB training data.  We used a basic Logistic Regression classifier to train two models: one on the original dataset and one on the EDA-augmented dataset.  The model trained on the original data had an accuracy of 77%, whereas the model trained on the augmented data had a much greater accuracy of 99% on the identical test set.
#This finding shows that EDA can significantly increase sentiment analysis performance by enriching the training data with syntactic and semantic changes.  While the performance increase is significant, we also identified possible concerns of overfitting or decreased data variety as a result of vigorous augmentation.